In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

from validation import mbe, nmbe, rmse, nrmse, get_summary_from_validation_metrics

In [2]:
data_path = Path.home()/"OneDrive-3E/RD_ResourceData/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis/IN3136_EURAC_shared_folder/data_sites"
results_path = Path.home()/"OneDrive-3E/RD_ResourceData/Research/Solar/tickets/2024/IN3043_accuracy_factors_analysis/IN3136_EURAC_shared_folder/validation"

## read sites metadata

In [3]:
sites_metadata = pd.read_csv(os.path.join(data_path, "sites_metadata.csv"), index_col=0)
sites_metadata

,country,latitude,longitude,altitude,provider
station_name,,,,,
camborne,United Kingdom,50.22,-5.32,88.0,bsrn
carpentras,France,44.08,5.06,100.0,bsrn
budapest_lorinc,Hungary,47.43,19.18,139.0,bsrn
magurele_mars,Romania,44.34,26.01,110.0,bsrn
cabauw,Netherlands,51.97,4.93,0.0,bsrn
...,...,...,...,...,...
dublin_arpt,Ireland,53.43,-6.23,82.0,wrdc
grossenzersdorf,Austria,48.20,16.57,157.0,wrdc
gonzaga,Italy,44.96,10.77,16.0,lombardy


## based on daily data

### read data

In [4]:
reference_data_daily = pd.read_csv(os.path.join(data_path, "parsed", "parsed_reference_data_daily.csv"), index_col=0)
model_data_daily = pd.read_csv(os.path.join(data_path, "parsed", "parsed_model_data_daily.csv"), index_col=0)

In [5]:
# convert index in pd.DateTimeIndex
reference_data_daily.index = pd.DatetimeIndex(reference_data_daily.index)
model_data_daily.index = pd.DatetimeIndex(model_data_daily.index)

### parse

In [6]:
# convert to nan where the other dataset is nan
reference_data_daily = reference_data_daily.where(model_data_daily.notna())
model_data_daily = model_data_daily.where(reference_data_daily.notna())

In [7]:
# aggregate
reference_data_monthly = reference_data_daily.resample('MS').sum(min_count=1)
model_data_monthly = model_data_daily.resample('MS').sum(min_count=1)
reference_data_yearly = reference_data_daily.resample('AS').sum(min_count=1)
model_data_yearly = model_data_daily.resample('AS').sum(min_count=1)

### calculate sites validation metrics

In [8]:
valid_data_pairs_daily = reference_data_daily.count()

In [9]:
validation_metrics = 100 * pd.DataFrame({site: {
    "nmbe": nmbe(model_data_daily[site], reference_data_daily[site]),
    "nrmse_yearly": nrmse(model_data_yearly[site], reference_data_yearly[site]),
    "nrmse_monthly": nrmse(model_data_monthly[site], reference_data_monthly[site]),
    "nrmse_daily": nrmse(model_data_daily[site], reference_data_daily[site]),
} for site in reference_data_daily.columns}).T    

In [10]:
validation_metrics["valid_data_pairs_daily"] = valid_data_pairs_daily

## based on hourly data

### read data

In [11]:
reference_data_hourly = pd.read_csv(os.path.join(data_path, "parsed", "parsed_reference_data_hourly.csv"), index_col=0)
model_data_hourly = pd.read_csv(os.path.join(data_path, "parsed", "parsed_model_data_hourly.csv"), index_col=0)

In [12]:
# convert index in pd.DateTimeIndex
reference_data_hourly.index = pd.DatetimeIndex(reference_data_hourly.index)
model_data_hourly.index = pd.DatetimeIndex(model_data_hourly.index)

### parse

In [13]:
# convert to nan where the other dataset is nan
reference_data_hourly = reference_data_hourly.where(model_data_hourly.notna())
model_data_hourly = model_data_hourly.where(reference_data_hourly.notna())

In [14]:
# aggregate
reference_data_daily = reference_data_hourly.resample('D').sum(min_count=1)
model_data_daily = model_data_hourly.resample('D').sum(min_count=1)
reference_data_monthly = reference_data_hourly.resample('MS').sum(min_count=1)
model_data_monthly = model_data_hourly.resample('MS').sum(min_count=1)
reference_data_yearly = reference_data_hourly.resample('AS').sum(min_count=1)
model_data_yearly = model_data_hourly.resample('AS').sum(min_count=1)

### calculate sites validation metrics

In [15]:
valid_data_pairs_hourly = reference_data_hourly.count()

In [16]:
validation_metrics_hourly = 100 * pd.DataFrame({site: {
    "nmbe": nmbe(model_data_hourly[site], reference_data_hourly[site]),
    "nrmse_yearly": nrmse(model_data_yearly[site], reference_data_yearly[site]),
    "nrmse_monthly": nrmse(model_data_monthly[site], reference_data_monthly[site]),
    "nrmse_daily": nrmse(model_data_daily[site], reference_data_daily[site]),
    "nrmse_hourly": nrmse(model_data_hourly[site], reference_data_hourly[site]),
} for site in reference_data_hourly.columns}).T

In [17]:
validation_metrics_hourly["valid_data_pairs_hourly"] = valid_data_pairs_hourly

## concat

In [18]:
validation_metrics = pd.concat([validation_metrics, validation_metrics_hourly], axis=0)

In [19]:
columns_round_2 = ["nmbe", "nrmse_yearly", "nrmse_monthly", "nrmse_daily", "nrmse_hourly"]

In [20]:
columns_int = ["valid_data_pairs_daily", "valid_data_pairs_hourly"]

In [21]:
for column in list(validation_metrics.columns):
    
    if column in columns_round_2:
        
        validation_metrics[column] = round(validation_metrics[column], 2)
    
    elif column in columns_int:
        
        validation_metrics[column] = validation_metrics[column].astype('Int64')

In [22]:
validation_metrics = validation_metrics[["nmbe", "nrmse_yearly", "nrmse_monthly", "nrmse_daily", "nrmse_hourly","valid_data_pairs_daily", "valid_data_pairs_hourly"]]

In [23]:
validation_metrics.to_csv(os.path.join(results_path, "validation_results_sites.csv"))

In [24]:
validation_metrics

,nmbe,nrmse_yearly,nrmse_monthly,nrmse_daily,nrmse_hourly,valid_data_pairs_daily,valid_data_pairs_hourly
aberporth,-5.52,5.53,6.57,11.95,NaN,720,<NA>
dunstaffnage,-4.73,4.77,5.99,16.45,NaN,646,<NA>
belfast_aldergrove_arpt,-1.47,1.57,2.94,8.37,NaN,721,<NA>
edinburgh,-1.43,1.54,2.91,9.98,NaN,647,<NA>
wattisham,-1.05,1.07,1.73,7.52,NaN,727,<NA>
...,...,...,...,...,...,...,...
weihenstephan_dürnast,2.68,2.85,4.55,9.48,17.04,<NA>,29941
saarbrücken_ensheim,0.79,1.28,2.54,7.94,15.87,<NA>,30742
gonzaga,-6.96,7.01,8.24,11.52,16.06,<NA>,28018
landriano,-0.32,1.42,3.18,7.71,12.99,<NA>,27945


## get summary values

In [25]:
validation_metrics_summary = get_summary_from_validation_metrics(validation_metrics)

In [26]:
validation_metrics_summary.to_csv(os.path.join(results_path, "validation_results_summary.csv"))

In [27]:
validation_metrics_summary

,nbr_of_sites,nbr_of_sites_hourly,nmbe_avg,nmbe_std,nrmse_yearly,nrmse_monthly,nrmse_daily,nrmse_hourly
summary,80,30,0.71,2.61,2.34,4.03,8.9,17.64
